In [ ]:
from common_preamble import *
from scipy import stats

In [ ]:
data_dir = f'{base_dir}/data'

In [ ]:
mm = trimesh_io.MeshMeta(disk_cache_path=mesh_dir,
                         cache_size=0, cv_path=mesh_cv_path)

ais_meshes = []
for oid in complete_ais_ids:
    ais_file = ais_mesh_dir + '/{}_ais.h5'.format(oid)
    if os.path.exists(ais_file):
        ais_mesh = mm.mesh(filename=ais_file)
        ais_meshes.append(ais_mesh)
    else:
        ais_meshes.append(None)
        print('{} AIS not found!'.format(oid))


In [ ]:
import cloudvolume as cv
svs = cv.CloudVolume(mesh_cv_path, mip=5, bounded=False, use_https=True)

In [ ]:
import cloudvolume as cv
from meshparty import skeletonize
def segmentation_mesh_zeros(mesh, dmax_vec, cv_path, mip=5, cap_y=True, use_sk=True, base_mip=0):
    svs = cv.CloudVolume(cv_path, mip=mip, bounded=False, use_https=True)
    print('\tComputing grid coordinates...')
    xmin, ymin, zmin = ((mesh.bounds[0] - max(dmax_vec)) / svs.resolution).astype(int)
    xmax, ymax, zmax = ((mesh.bounds[1] + max(dmax_vec)) / svs.resolution).astype(int)
    if cap_y:
        ymin = (mesh.bounds[0][1] / svs.resolution[1]).astype(int)
        ymax = (mesh.bounds[1][1] / svs.resolution[1]).astype(int)

    X, Y, Z = np.meshgrid(np.arange(xmin, xmax), np.arange(ymin,ymax), np.arange(zmin, zmax), indexing='ij')

    xyzs = np.vstack([X.ravel(), Y.ravel(), Z.ravel()]).T * svs.resolution
    if use_sk:
        sk = skeletonize.skeletonize_mesh(mesh)
        vert_obj = sk
    else:
        vert_obj = mesh
        
    try:
        dsp, _ = vert_obj.pykdtree.query(xyzs, distance_upper_bound=max(dmax_vec))
    except:
        dsp, _ = vert_obj.kdtree.query(xyzs, distance_upper_bound=max(dmax_vec))

    print('\tDownloading segmentation...')

    xslice = slice(xmin, xmax)
    yslice = slice(ymin, ymax)
    zslice = slice(zmin, zmax)
    
    sv_dat = np.squeeze(svs[xslice, yslice, zslice])
    
    print('\tComputing proximate grid volume...')
    outside_seg = (sv_dat==0).astype(int).ravel()
    frac_out = []
    for dmax in dmax_vec:
        valid=dsp<dmax
        vox_outside = np.sum(outside_seg[valid])
        vox_total = np.sum(valid)
        frac_out.append(vox_outside / vox_total)
    return frac_out

In [ ]:
old_version = 183
fraction_zeros_old_df = pd.read_hdf(data_dir + '/mask_fraction_data_v{}.hdf'.format(old_version),
                                    'mask_fraction_df')

In [ ]:
dmax =  [5000, 7500, 10000, 15000]
mip=5
cap_y=True
f_all = {}
for oid, ais_mesh in tqdm.tqdm(zip(complete_ais_ids, ais_meshes), total=len(ais_meshes)):
    if np.any(fraction_zeros_old_df['root_id']==oid):
        f = fraction_zeros_old_df[fraction_zeros_old_df['root_id']==oid].values[0,1:]
    else:
        f = segmentation_mesh_zeros(ais_mesh, dmax, mesh_cv_path, mip, cap_y)
    f_all[oid] = f

    
d_dict = {k:{} for k in dmax}
for k, x in f_all.items():
    for ii, d in enumerate(dmax):
        d_dict[d][k] = float(x[ii])

        
df_cols = {'d_{}'.format(d): d_dict[d] for d in dmax}
mask_fraction_df = pd.DataFrame(df_cols).reset_index().rename(columns={'index':'root_id'})
mask_fraction_df.to_hdf(data_dir + '/mask_fraction_data_v{}.hdf'.format(data_version),
                        'mask_fraction_df')

In [ ]:
1